# Доля аудитории каналов за день и в прайм-тайм
Пример расчета доли телеканалов по нескольким ЦА за полный день и в прайм-тайм

## Описание задачи и условий расчета
- Период: 03.10.2022 - 09.10.2022
- Временной интервал: 05:00-29:00 и 19:00-23:00
- ЦА: Все 4+ Россия 100+, Все 18-54 Россия 100+
- Место просмотра: Все места (дом+дача)
- Каналы: все каналы проекта TVI									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['100+', 'возраст'])

Запрошены записи: 0 - 127
Всего найдено записей: 127



,id,valueId,name,colName,valueName
0,2,33,Возраст,Age,33
1,16,1,Возраст детей: До 1 года,KidsAge2,Да
2,16,0,Возраст детей: До 1 года,KidsAge2,Нет
3,20,0,Возраст детей: 7-11 лет,KidsAge6,Нет
4,20,1,Возраст детей: 7-11 лет,KidsAge6,Да
...,...,...,...,...,...
122,2,10,Возраст,Age,10
123,2,89,Возраст,Age,89
124,2,8,Возраст,Age,8
125,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [3]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-10-03', '2022-10-09')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группы ЦА и временных интервалов

In [4]:
# Формируем необходимые группы ЦА 
targets = {
    '4+ Россия 100+':'cube100Plus100Minus = 1',
    '18-54 Россия 100+':'age >= 18 AND age <= 54 AND cube100Plus100Minus = 1'
}

# Формируем необходимые группы временных интервалов 
timebands = {
    'total day':'timeBand1 >= 50000 AND timeBand1 < 290000',
    '19:00 - 23:00':'timeBand1 >= 190000 AND timeBand1 < 230000'
}

## Расчет задания

In [5]:
# Посчитаем комбинации ЦА и временных интервалов в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы формируем задания и отправляем на расчет
for target, tar_log in targets.items():
    for timeband, tb_log in timebands.items():    
        # Формируем задание на расчет
        project_name = f"{target}&{timeband}" 
        basedemo_filter = tar_log
        time_filter = tb_log

        # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
        task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name    
        tsk['task'] = mtask.send_timeband_task(task_json)
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Отправляем задания на расчет
....
Ждем выполнения
Расчет задач (4) [ = = = = ] время расчета: 0:00:18.824783
Расчет завершен, получаем результат
Собираем таблицу
....

,prj_name,tvNetName,Share
0,4+ Россия 100+&total day,ТВ ЦЕНТР,4.034976
1,4+ Россия 100+&total day,РЕН ТВ,4.843386
2,4+ Россия 100+&total day,МИР,1.709996
3,4+ Россия 100+&total day,ИЗМЕРЯЕМОЕ ТЕМАТИЧЕСКОЕ ТВ,17.542196
4,4+ Россия 100+&total day,Ю,0.684260
...,...,...,...
28,18-54 Россия 100+&19:00 - 23:00,ПЯТЫЙ КАНАЛ,4.490567
29,18-54 Россия 100+&19:00 - 23:00,СПАС,0.218886
30,18-54 Россия 100+&19:00 - 23:00,ЗВЕЗДА,1.205488
31,18-54 Россия 100+&19:00 - 23:00,OTHER TV SET,27.623965


In [6]:
# Разделяем колонку с именем проекта на ЦА и временной интервал
df[['TA', 'Timeband']] = df['prj_name'].str.split('&', 1, expand=True)

# Удаляем исходную колонку
df = df.drop(columns=['prj_name'])

# Разворачиваем таблицу
df_res = pd.pivot_table(df, values=['Share'],
                        index=['tvNetName'], 
                        columns=['Timeband','TA'])
df_res

Share                                   \
Timeband                       19:00 - 23:00                        total day   
TA                         18-54 Россия 100+ 4+ Россия 100+ 18-54 Россия 100+   
tvNetName                                                                       
2X2                                 0.391857       0.230098          0.383336   
OTHER TV SET                       27.623965      18.744359         27.668773   
ДОМ КИНО                            1.158787       0.846784          1.251629   
ДОМАШНИЙ                            3.051049       4.119854          3.110761   
ЗВЕЗДА                              1.205488       1.864392          1.137051   
ИЗМЕРЯЕМОЕ ЛОКАЛЬНОЕ ТВ             2.708715       3.127310          2.583626   
ИЗМЕРЯЕМОЕ ТЕМАТИЧЕСКОЕ ТВ         17.002236      14.475136         20.832579   
КАРУСЕЛЬ                            2.830774       2.727299          2.912847   
МАТЧ ТВ                             1.481678       1.242768          1.189032   
МИР                                 1.147139       1.677030          1.251318   
МУЗ ТВ                              0.721288       0.442465          0.906142   
МУЛЬТ                               0.545043       0.595563          0.950307   
НТВ                                 7.699617      10.848436          6.288315   
ПЕРВЫЙ КАНАЛ                        8.705378       9.794340          8.405424   
ПЯТНИЦА                             5.009358       2.940992          4.023755   
ПЯТЫЙ КАНАЛ                         4.490567       6.207815          4.317362   
РЕКЛАМНЫЕ КАНАЛЫ                    0.507478       0.382445          0.577649   
РЕН ТВ                              5.658260       3.993636          6.154580   
РОССИЯ 1                           11.574872      17.527963         10.104958   
РОССИЯ 24                           2.084299       1.945201          3.086135   
РОССИЯ К                            0.464191       0.753380          0.514804   
СОЛНЦЕ                              1.135523       1.259802          1.025183   
СОЛОВЬЁВLIVE                        0.177904       0.231550          0.247352   
СПАС                                0.218886       0.361176          0.271487   
СТС                                 9.270576       5.447223          7.567775   
СТС LOVE                            0.573419       0.352726          0.596575   
СУББОТА                             0.440769       0.242093          0.771801   
ТВ ЦЕНТР                            2.376255       4.916908          2.129176   
ТВ-3                                3.320803       2.148098          3.771746   
ТНТ                                 6.986188       3.878084          5.642470   
ТНТ 4                               0.871091       0.472242          0.872902   
ЧЕ                                  0.660090       0.558411          0.657990   
Ю                                   0.825423       0.659122          0.987675   

                                           
Timeband                                   
TA                         4+ Россия 100+  
tvNetName                                  
2X2                              0.235108  
OTHER TV SET                    18.505616  
ДОМ КИНО                         0.879254  
ДОМАШНИЙ                         3.746776  
ЗВЕЗДА                           1.731364  
ИЗМЕРЯЕМОЕ ЛОКАЛЬНОЕ ТВ          3.138313  
ИЗМЕРЯЕМОЕ ТЕМАТИЧЕСКОЕ ТВ      17.542196  
КАРУСЕЛЬ                         2.671496  
МАТЧ ТВ                          1.056132  
МИР                              1.709996  
МУЗ ТВ                           0.522673  
МУЛЬТ                            0.854186  
НТВ                              9.446152  
ПЕРВЫЙ КАНАЛ                     9.360335  
ПЯТНИЦА                          2.332238  
ПЯТЫЙ КАНАЛ                      6.208600  
РЕКЛАМНЫЕ КАНАЛЫ                 0.437622  
РЕН ТВ                           4.843386  
РОССИЯ 1                        15.931729  
РОССИЯ 24                    

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [7]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('07_share_totalday_primetime'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()